In [1]:
import numpy as np  
from matplotlib import pyplot as plt  
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import torch
import torch.nn.functional as nn
import torch.autograd as autograd
from torch import optim
from torch.autograd import Variable
import torch.nn.functional as F
import os
import matplotlib.gridspec as gridspec
import os
from tqdm import tqdm
%matplotlib inline 

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import data_utils

In [4]:
data = data_utils.load_training_data()
data.shape

(7352, 128)

In [5]:
# system parameters
n_epochs = 500
batch_size = 10
lr = 0.001 # adam:learning rate
b1 = 0.9 # adam: decay of first order momentum of gradient
b2 = 0.999 # adam: decay of first order momentum of gradient
img_size = 28 # size of each image dimension
channels = 1 # number of image channels

alpha= 10
Dim = 10
p_miss = 0.5
p_hint = 0.9
data_loader = data_utils.DataLoader(data=data,batch_size=batch_size, num_steps=Dim)

cuda = True if torch.cuda.is_available() else False
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [6]:
X_mb,_ = data_loader.next_batch()

In [7]:
X_mb.shape

(10, 10)

In [8]:
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / np.sqrt(in_dim / 2.)
    return Variable(torch.randn(*size) * xavier_stddev, requires_grad=True)

In [9]:
""" ==================== DISCRIMINATOR ======================== """
D_W1 = xavier_init(size = [Dim*2, 256])     # Data + Hint as inputs
D_b1 = Variable(torch.zeros(256),requires_grad=True)

D_W2 = xavier_init(size = [256, 128])
D_b2 = Variable(torch.zeros(128),requires_grad=True)

D_W3 = xavier_init(size = [128, Dim])
D_b3 = Variable(torch.zeros(Dim),requires_grad=True)  

def discriminator(x, m, g, h):
    inp = m * x + (1-m) * g  # Replace missing values to the imputed values
    inputs = torch.cat([inp,h],1)  # Hint + Data Concatenate
    D_h1 = nn.relu(torch.matmul(inputs, D_W1) + D_b1)
    D_h2 = nn.relu(torch.matmul(D_h1, D_W2) + D_b2)
    D_logit = torch.matmul(D_h2, D_W3) + D_b3
    D_prob = nn.sigmoid(D_logit)  # [0,1] Probability Output
    
    return D_prob

In [10]:
""" ==================== GENERATOR ======================== """
G_W1 = xavier_init(size = [Dim*2, 256])    # Data + Mask as inputs (Random Noises are in Missing Components)
G_b1 = Variable(torch.zeros(256),requires_grad=True)

G_W2 = xavier_init(size = [256, 128])
G_b2 = Variable(torch.zeros(128),requires_grad=True)

G_W3 = xavier_init(size = [128, Dim])
G_b3 = Variable(torch.zeros(Dim),requires_grad=True)

def generator(x,z,m):
    inp = m * x + (1-m) * z  # Fill in random noise on the missing values
    inputs = torch.cat([inp,m],1)  # Mask + Data Concatenate
    G_h1 = nn.relu(torch.matmul(inputs, G_W1) + G_b1)
    G_h2 = nn.relu(torch.matmul(G_h1, G_W2) + G_b2)
    G_prob = nn.sigmoid(torch.matmul(G_h2, G_W3) + G_b3) # [0,1] normalized Output
    
    return G_prob

In [11]:
G_params = [G_W1, G_W2, G_W3, G_b1, G_b2, G_b3]
D_params = [D_W1, D_W2, D_W3, D_b1, D_b2, D_b3]
params = G_params + D_params

In [12]:
""" ===================== TRAINING ======================== """
def plot(samples):
    fig = plt.figure(figsize = (5,5))
    gs = gridspec.GridSpec(5,5)
    gs.update(wspace=0.05, hspace=0.05)
    
    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28,28), cmap='Greys_r')
        
    return fig
def reset_grad():
    for p in params:
        if p.grad is not None:
            data = p.grad.data
            p.grad = Variable(data.new().resize_as_(data).zero_())
            
G_solver = optim.Adam(G_params, lr=1e-3)
D_solver = optim.Adam(D_params, lr=1e-3)

def sample_Z(m, n):
    return np.random.uniform(0., 1., size = [m, n])        

# Mask Vector and Hint Vector Generation
def sample_M(m, n, p):
    A = np.random.uniform(0., 1., size = [m, n])
    B = A > p
    C = 1.*B
    return C
# make output file
if not os.path.exists('MNIST_Impuation_output/'):
    os.makedirs('MNIST_Impuation_output/')
i = 1
for it in tqdm(range(n_epochs)):
    data_loader.reset()
    while data_loader.has_next():
        X_mb,_ = data_loader.next_batch()

        X_mb = Variable(torch.from_numpy(X_mb.astype('float32')))
        Z_mb = sample_Z(batch_size, Dim) 
        Z_mb = Variable(torch.from_numpy(Z_mb.astype('float32')))
        M_mb = sample_M(batch_size, Dim, p_miss)
        M_mb = Variable(torch.from_numpy(M_mb.astype('float32')))
        H_mb1 = sample_M(batch_size, Dim, 1-p_hint)
        H_mb1 = Variable(torch.from_numpy(H_mb1.astype('float32')))
        H_mb = M_mb * H_mb1

        New_X_mb = M_mb * X_mb + (1-M_mb) * Z_mb  # Missing Data Introduce
        # Dicriminator forward-loss-backward-update
        G_sample = generator(X_mb,Z_mb,M_mb)
        D_sample = discriminator(X_mb, M_mb, G_sample, H_mb)
        D_loss = -torch.mean(M_mb * torch.log(D_sample + 1e-8) + (1-M_mb) * torch.log(1. - D_sample + 1e-8)) * 2
        D_loss.backward()
        D_solver.step()
        reset_grad()

        # Generator forward-loss-backward-update
        G_sample = generator(X_mb,Z_mb,M_mb)
        D_sample = discriminator(X_mb, M_mb, G_sample, H_mb)
        G_loss1 = -torch.mean((1-M_mb) * torch.log(D_sample + 1e-8)) / torch.mean(1-M_mb)
        MSE_train_loss = torch.mean((M_mb * X_mb - M_mb * G_sample)**2) / torch.mean(M_mb)
        MSE_test_loss = torch.mean(((1-M_mb) * X_mb - (1-M_mb)*G_sample)**2) / torch.mean(1-M_mb)
        G_loss = G_loss1  + alpha * MSE_train_loss 
        G_loss.backward()
        G_solver.step()
        reset_grad()
    
    # Print and Plot
    
 
    
        
    if it % 10 == 0:
        print('Iter: {}'.format(it))
        print('Train_loss: {:.4}'.format(MSE_train_loss))
        print('Test_loss: {:.4}'.format(MSE_test_loss))
        print()
        



  0%|          | 1/500 [00:02<18:52,  2.27s/it]

Iter: 0
Train_loss: 0.0312
Test_loss: 0.1131



  2%|▏         | 11/500 [00:28<21:09,  2.60s/it]

Iter: 10
Train_loss: 0.02722
Test_loss: 0.07947



  4%|▍         | 21/500 [00:56<21:28,  2.69s/it]

...

Iter: 140
Train_loss: 0.03703
Test_loss: 0.08939



 30%|███       | 151/500 [09:35<22:10,  3.81s/it]

Iter: 150
Train_loss: 0.04207
Test_loss: 0.07288



 30%|███       | 152/500 [09:40<22:08,  3.82s/it]

KeyboardInterrupt: 

In [29]:
def reset_grad():
    for p in params:
        if p.grad is not None:
            data = p.grad.data
            p.grad = Variable(data.new().resize_as_(data).zero_())
            
G_solver = optim.Adam(G_params, lr=1e-3)
D_solver = optim.Adam(D_params, lr=1e-3)

def sample_Z(m, n):
    return np.random.uniform(0., 1., size = [m, n])        

# Mask Vector and Hint Vector Generation
def sample_M(m, n, p):
    A = np.random.uniform(0., 1., size = [m, n])
    B = A > p
    C = 1.*B
    return C
# make output file
if not os.path.exists('MNIST_Impuation_output/'):
    os.makedirs('MNIST_Impuation_output/')
i = 1
for it in tqdm(range(n_epochs)):
    data_loader.reset()
    while data_loader.has_next():
        X_mb,_ = data_loader.next_batch()

        X_mb = Variable(torch.from_numpy(X_mb.astype('float32')))
        Z_mb = sample_Z(batch_size, Dim) 
        Z_mb = Variable(torch.from_numpy(Z_mb.astype('float32')))
        M_mb = sample_M(batch_size, Dim, p_miss)
        M_mb = Variable(torch.from_numpy(M_mb.astype('float32')))
        H_mb1 = sample_M(batch_size, Dim, 1-p_hint)
        H_mb1 = Variable(torch.from_numpy(H_mb1.astype('float32')))
        H_mb = M_mb * H_mb1

        New_X_mb = M_mb * X_mb + (1-M_mb) * Z_mb  # Missing Data Introduce
        # Dicriminator forward-loss-backward-update
        G_sample = generator(X_mb,Z_mb,M_mb)
        D_sample = discriminator(X_mb, M_mb, G_sample, H_mb)
        D_loss = -torch.mean(M_mb * torch.log(D_sample + 1e-8) + (1-M_mb) * torch.log(1. - D_sample + 1e-8)) * 2
        D_loss.backward()
        D_solver.step()
        reset_grad()

        # Generator forward-loss-backward-update
        G_sample = generator(X_mb,Z_mb,M_mb)
        D_sample = discriminator(X_mb, M_mb, G_sample, H_mb)
        G_loss1 = -torch.mean((1-M_mb) * torch.log(D_sample + 1e-8)) / torch.mean(1-M_mb)
        MSE_train_loss = torch.mean((M_mb * X_mb - M_mb * G_sample)**2) / torch.mean(M_mb)
        MSE_test_loss = torch.mean(((1-M_mb) * X_mb - (1-M_mb)*G_sample)**2) / torch.mean(1-M_mb)
        G_loss = G_loss1  + alpha * MSE_train_loss 
        G_loss.backward()
        G_solver.step()
        reset_grad()
    
    # Print and Plot
    
 
    
        
    if it % 10 == 0:
        print('Iter: {}'.format(it))
        print('Train_loss: {:.4}'.format(MSE_train_loss))
        print('Test_loss: {:.4}'.format(MSE_test_loss))
        print()
        




  1%|          | 3/500 [00:00<00:22, 21.88it/s]

Iter: 0
Train_loss: 0.02545
Test_loss: 0.03415



  3%|▎         | 16/500 [00:00<00:21, 22.63it/s]

Iter: 10
Train_loss: 0.007392
Test_loss: 0.1816



  5%|▌         | 25/500 [00:01<00:19, 23.96it/s]

...

 96%|█████████▋| 482/500 [00:28<00:01, 17.00it/s]

Iter: 480
Train_loss: 0.0004323
Test_loss: 0.003546



 99%|█████████▉| 494/500 [00:29<00:00, 16.93it/s]

Iter: 490
Train_loss: 0.0004038
Test_loss: 0.0005646



100%|██████████| 500/500 [00:29<00:00, 16.95it/s]


In [12]:
n_epochs = 1000
def reset_grad():
    for p in params:
        if p.grad is not None:
            data = p.grad.data
            p.grad = Variable(data.new().resize_as_(data).zero_())
            
G_solver = optim.Adam(G_params, lr=1e-3)
D_solver = optim.Adam(D_params, lr=1e-3)

def sample_Z(m, n):
    return np.random.uniform(0., 1., size = [m, n])        

# Mask Vector and Hint Vector Generation
def sample_M(m, n, p):
    A = np.random.uniform(0., 1., size = [m, n])
    B = A > p
    C = 1.*B
    return C
# make output file
if not os.path.exists('MNIST_Impuation_output/'):
    os.makedirs('MNIST_Impuation_output/')
i = 1
for it in tqdm(range(n_epochs)):
    data_loader.reset()
    while data_loader.has_next():
        X_mb,_ = data_loader.next_batch()

        X_mb = Variable(torch.from_numpy(X_mb.astype('float32')))
        Z_mb = sample_Z(batch_size, Dim) 
        Z_mb = Variable(torch.from_numpy(Z_mb.astype('float32')))
        M_mb = sample_M(batch_size, Dim, p_miss)
        M_mb = Variable(torch.from_numpy(M_mb.astype('float32')))
        H_mb1 = sample_M(batch_size, Dim, 1-p_hint)
        H_mb1 = Variable(torch.from_numpy(H_mb1.astype('float32')))
        H_mb = M_mb * H_mb1

        New_X_mb = M_mb * X_mb + (1-M_mb) * Z_mb  # Missing Data Introduce
        # Dicriminator forward-loss-backward-update
        G_sample = generator(X_mb,Z_mb,M_mb)
        D_sample = discriminator(X_mb, M_mb, G_sample, H_mb)
        D_loss = -torch.mean(M_mb * torch.log(D_sample + 1e-8) + (1-M_mb) * torch.log(1. - D_sample + 1e-8)) * 2
        D_loss.backward()
        D_solver.step()
        reset_grad()

        # Generator forward-loss-backward-update
        G_sample = generator(X_mb,Z_mb,M_mb)
        D_sample = discriminator(X_mb, M_mb, G_sample, H_mb)
        G_loss1 = -torch.mean((1-M_mb) * torch.log(D_sample + 1e-8)) / torch.mean(1-M_mb)
        MSE_train_loss = torch.mean((M_mb * X_mb - M_mb * G_sample)**2) / torch.mean(M_mb)
        MSE_test_loss = torch.mean(((1-M_mb) * X_mb - (1-M_mb)*G_sample)**2) / torch.mean(1-M_mb)
        G_loss = G_loss1  + alpha * MSE_train_loss 
        G_loss.backward()
        G_solver.step()
        reset_grad()
    
    # Print and Plot
    
 
    
        
    if it % 10 == 0:
        print('Iter: {}'.format(it))
        print('Train_loss: {:.4}'.format(MSE_train_loss))
        print('Test_loss: {:.4}'.format(MSE_test_loss))
        print()
        

  0%|          | 5/1000 [00:00<00:46, 21.42it/s]

Iter: 0
Train_loss: 0.0188
Test_loss: 0.04297



  1%|▏         | 14/1000 [00:00<00:44, 22.15it/s]

Iter: 10
Train_loss: 0.003816
Test_loss: 0.1588



  3%|▎         | 26/1000 [00:01<00:40, 23.77it/s]

...

 98%|█████████▊| 984/1000 [01:03<00:01, 15.46it/s]

Iter: 980
Train_loss: 0.000406
Test_loss: 0.000437



 99%|█████████▉| 992/1000 [01:04<00:00, 15.45it/s]

Iter: 990
Train_loss: 0.0004182
Test_loss: 0.0004216



100%|██████████| 1000/1000 [01:04<00:00, 15.45it/s]
